##  The NIST Special Publication 800-63B
<p>If you – 50 years ago – needed to come up with a secret password you were probably part of a secret espionage organization or (more likely) you were pretending to be a spy when playing as a kid. Today, many of us are forced to come up with new passwords <em>all the time</em> when signing into sites and apps. As a password <em>inventeur</em> it is your responsibility to come up with good, hard-to-crack passwords. But it is also in the interest of sites and apps to make sure that you use good passwords. The problem is that it's really hard to define what makes a good password. However, <em>the National Institute of Standards and Technology</em> (NIST) knows what the second best thing is: To make sure you're at least not using a <em>bad</em> password. </p>
<p>In this notebook, we will go through the rules in <a href="https://pages.nist.gov/800-63-3/sp800-63b.html">NIST Special Publication 800-63B</a> which details what checks a <em>verifier</em> (what the NIST calls a second party responsible for storing and verifying passwords) should perform to make sure users don't pick bad passwords. We will go through the passwords of users from a fictional company and use R to flag the users with bad passwords. But us being able to do this already means the fictional company is breaking one of the rules of 800-63B:</p>
<blockquote>
  <p>Verifiers SHALL store memorized secrets in a form that is resistant to offline attacks. Memorized secrets SHALL be salted and hashed using a suitable one-way key derivation function.</p>
</blockquote>
<p>That is, never save users' passwords in plaintext, always encrypt the passwords! Keeping this in mind for the next time we're building a password management system, let's load in the data.</p>
<p><em>Warning: The list of passwords and the fictional user database both contain <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have not been filtered in any way and include words that are explicit, derogatory and offensive.</em></p>

In [53]:
# Importing the tidyverse library
library(tidyverse)

# Loading in datasets/users.csv 
users <- read_csv("datasets/users.csv")

# Counting how many users we've got
count(users)
# Taking a look at the 12 first users
head(users,12)

n
<int>
982


,id,user_name,password
,<int>,<chr>,<chr>
1,1,vance.jennings,joobheco
2,2,consuelo.eaton,0869347314
3,3,mitchel.perkins,fabypotter
4,4,odessa.vaughan,aharney88
5,5,araceli.wilder,acecdn3000
6,6,shawn.harrington,5278049
7,7,evelyn.gay,master
8,8,noreen.hale,murphy
9,9,gladys.ward,lwsves2


##  Passwords should not be too short
<p>If we take a look at the first 12 users above we already see some bad passwords. But let's not get ahead of ourselves and start flagging passwords <em>manually</em>. What is the first thing we should check according to the NIST Special Publication 800-63B?</p>
<blockquote>
  <p>Verifiers SHALL require subscriber-chosen memorized secrets to be at least 8 characters in length.</p>
</blockquote>
<p>Ok, so the passwords of our users shouldn't be too short. Let's start by checking that!</p>

In [55]:
# Calculating the lengths of users' passwords
users$length <- str_length(users$password)

# Flagging the users with too short passwords
users$too_short <- users$length < 8

# Counting the number of users with too short passwords
sum(users$too_short)
# Taking a look at the 12 first rows
head(users,12)

[1] 376

,id,user_name,password,length,too_short
,<int>,<chr>,<chr>,<int>,<lgl>
1,1,vance.jennings,joobheco,8,FALSE
2,2,consuelo.eaton,0869347314,10,FALSE
3,3,mitchel.perkins,fabypotter,10,FALSE
4,4,odessa.vaughan,aharney88,9,FALSE
5,5,araceli.wilder,acecdn3000,10,FALSE
6,6,shawn.harrington,5278049,7,TRUE
7,7,evelyn.gay,master,6,TRUE
8,8,noreen.hale,murphy,6,TRUE
9,9,gladys.ward,lwsves2,7,TRUE


##   Common passwords people use
<p>Already this simple rule flagged a couple of offenders among the first 12 users. Next up in Special Publication 800-63B is the rule that</p>
<blockquote>
  <p>verifiers SHALL compare the prospective secrets against a list that contains values known to be commonly-used, expected, or compromised.</p>
  <ul>
  <li>Passwords obtained from previous breach corpuses.</li>
  <li>Dictionary words.</li>
  <li>Repetitive or sequential characters (e.g. ‘aaaaaa’, ‘1234abcd’).</li>
  <li>Context-specific words, such as the name of the service, the username, and derivatives thereof.</li>
  </ul>
</blockquote>
<p>We're going to check these in order and start with <em>Passwords obtained from previous breach corpuses</em>, that is, websites where hackers have leaked all the users' passwords. As many websites don't follow the NIST guidelines and encrypt passwords there now exist large lists of the most popular passwords. Let's start by loading in the 10,000 most common passwords which I've taken from <a href="https://github.com/danielmiessler/SecLists/tree/master/Passwords">here</a>.</p>

In [57]:
# Reading in the top 10000 passwords
common_passwords <- read_lines('datasets/10_million_password_list_top_10000.txt')

# Taking a look at the top 100
head(common_passwords,100)

[1] "123456"     "password"   "12345678"   "qwerty"     "123456789" 
  [6] "12345"      "1234"       "111111"     "1234567"    "dragon"    
 [11] "123123"     "baseball"   "abc123"     "football"   "monkey"    
 [16] "letmein"    "696969"     "shadow"     "master"     "666666"    
 [21] "qwertyuiop" "123321"     "mustang"    "1234567890" "michael"   
 [26] "654321"     "pussy"      "superman"   "1qaz2wsx"   "7777777"   
 [31] "fuckyou"    "121212"     "000000"     "qazwsx"     "123qwe"    
 [36] "killer"     "trustno1"   "jordan"     "jennifer"   "zxcvbnm"   
 [41] "asdfgh"     "hunter"     "buster"     "soccer"     "harley"    
 [46] "batman"     "andrew"     "tigger"     "sunshine"   "iloveyou"  
 [51] "fuckme"     "2000"       "charlie"    "robert"     "thomas"    
 [56] "hockey"     "ranger"     "daniel"     "starwars"   "klaster"   
 [61] "112233"     "george"     "asshole"    "computer"   "michelle"  
 [66] "jessica"    "pepper"     "1111"       "zxcvbn"     "555555"    
 [71] "11111111"   "131313"     "freedom"    "777777"     "pass"      
 [76] "fuck"       "maggie"     "159753"     "aaaaaa"     "ginger"    
 [81] "princess"   "joshua"     "cheese"     "amanda"     "summer"    
 [86] "love"       "ashley"     "6969"       "nicole"     "chelsea"   
 [91] "biteme"     "matthew"    "access"     "yankees"    "987654321" 
 [96] "dallas"     "austin"     "thunder"    "taylor"     "matrix"

##   Passwords should not be common passwords
<p>The list of passwords was ordered, with the most common passwords first, and so we shouldn't be surprised to see passwords like <code>123456</code> and <code>qwerty</code> above. As hackers also have access to this list of common passwords, it's important that none of our users use these passwords!</p>
<p>Let's flag all the passwords in our user database that are among the top 10,000 used passwords.</p>

In [59]:
# Flagging the users with passwords that are common passwords
users$common_password <- users$passwor %in% common_passwords

# Counting the number of users using common passwords
sum(users$common_password)
# Taking a look at the 12 first rows
head(users,12)

[1] 129

,id,user_name,password,length,too_short,common_password
,<int>,<chr>,<chr>,<int>,<lgl>,<lgl>
1,1,vance.jennings,joobheco,8,FALSE,FALSE
2,2,consuelo.eaton,0869347314,10,FALSE,FALSE
3,3,mitchel.perkins,fabypotter,10,FALSE,FALSE
4,4,odessa.vaughan,aharney88,9,FALSE,FALSE
5,5,araceli.wilder,acecdn3000,10,FALSE,FALSE
6,6,shawn.harrington,5278049,7,TRUE,FALSE
7,7,evelyn.gay,master,6,TRUE,TRUE
8,8,noreen.hale,murphy,6,TRUE,TRUE
9,9,gladys.ward,lwsves2,7,TRUE,FALSE


##  Passwords should not be common words
<p>Ay ay ay! It turns out many of our users use common passwords, and of the first 12 users there are already two. However, as most common passwords also tend to be short, they were already flagged as being too short. What is the next thing we should check?</p>
<blockquote>
  <p>Verifiers SHALL compare the prospective secrets against a list that contains […] dictionary words.</p>
</blockquote>
<p>This follows the same logic as before: It is easy for hackers to check users' passwords against common English words and therefore common English words make bad passwords. Let's check our users' passwords against the top 10,000 English words from <a href="https://github.com/first20hours/google-10000-english">Google's Trillion Word Corpus</a>.</p>

In [61]:
# Reading in a list of the 10000 most common words
words <- read_lines("datasets/google-10000-english.txt")

# Flagging the users with passwords that are common words
users$common_word <- users$password %in% words

# Counting the number of users using common words as passwords
sum(users$common_word)
# Taking a look at the 12 first rows
head(users,12)

[1] 136

,id,user_name,password,length,too_short,common_password,common_word
,<int>,<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>
1,1,vance.jennings,joobheco,8,FALSE,FALSE,FALSE
2,2,consuelo.eaton,0869347314,10,FALSE,FALSE,FALSE
3,3,mitchel.perkins,fabypotter,10,FALSE,FALSE,FALSE
4,4,odessa.vaughan,aharney88,9,FALSE,FALSE,FALSE
5,5,araceli.wilder,acecdn3000,10,FALSE,FALSE,FALSE
6,6,shawn.harrington,5278049,7,TRUE,FALSE,FALSE
7,7,evelyn.gay,master,6,TRUE,TRUE,TRUE
8,8,noreen.hale,murphy,6,TRUE,TRUE,TRUE
9,9,gladys.ward,lwsves2,7,TRUE,FALSE,FALSE


##  Passwords should not be your name
<p>It turns out many of our passwords were common English words too! Next up on the NIST list:</p>
<blockquote>
  <p>Verifiers SHALL compare the prospective secrets against a list that contains […] context-specific words, such as the name of the service, the username, and derivatives thereof.</p>
</blockquote>
<p>Ok, so there are many things we could check here. One thing to notice is that our users' usernames consist of their first names and last names separated by a dot. For now, let's just flag passwords that are the same as either a user's first or last name.</p>

In [63]:
# Extracting first and last names into their own columns
users$first_name <- str_extract(users$user_name, "^\\w+") 
users$last_name <- str_extract(users$user_name, "\\w+$")

# Flagging the users with passwords that matches their names
users$uses_name <- str_to_lower(users$password) == users$first_name |
                   str_to_lower(users$password) == users$last_name
# Counting the number of users using names as passwords
sum(users$uses_name)
# Taking a look at the 12 first rows
head(users,12)

[1] 50

,id,user_name,password,length,too_short,common_password,common_word,first_name,last_name,uses_name
,<int>,<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<lgl>
1,1,vance.jennings,joobheco,8,FALSE,FALSE,FALSE,vance,jennings,FALSE
2,2,consuelo.eaton,0869347314,10,FALSE,FALSE,FALSE,consuelo,eaton,FALSE
3,3,mitchel.perkins,fabypotter,10,FALSE,FALSE,FALSE,mitchel,perkins,FALSE
4,4,odessa.vaughan,aharney88,9,FALSE,FALSE,FALSE,odessa,vaughan,FALSE
5,5,araceli.wilder,acecdn3000,10,FALSE,FALSE,FALSE,araceli,wilder,FALSE
6,6,shawn.harrington,5278049,7,TRUE,FALSE,FALSE,shawn,harrington,FALSE
7,7,evelyn.gay,master,6,TRUE,TRUE,TRUE,evelyn,gay,FALSE
8,8,noreen.hale,murphy,6,TRUE,TRUE,TRUE,noreen,hale,FALSE
9,9,gladys.ward,lwsves2,7,TRUE,FALSE,FALSE,gladys,ward,FALSE


##  Passwords should not be repetitive
<p>Milford Hubbard (user number 12 above), what where you thinking!? Ok, so the last thing we are going to check is a bit tricky:</p>
<blockquote>
  <p>verifiers SHALL compare the prospective secrets [so that they don't contain] repetitive or sequential characters (e.g. ‘aaaaaa’, ‘1234abcd’).</p>
</blockquote>
<p>This is tricky to check because what is <em>repetitive</em> is hard to define. Is <code>11111</code> repetitive? Yes! Is <code>12345</code> repetitive? Well, kind of. Is <code>13579</code> repetitive? Maybe not..? To check for <em>repetitiveness</em> can be arbitrarily complex, but here we're only going to do something simple. We're going to flag all passwords that contain 4 or more repeated characters.</p>

In [65]:
# Splitting the passwords into vectors of single characters
split_passwords <- str_split(users$password,"")

# Picking out the max number of repeat characters for each password
users$max_repeats <- sapply(split_passwords, function(split_password) {
        rle_password <- rle(split_password)
        max(rle_password$lengths)
})


# Flagging the passwords with >= 4 repeats
users$too_many_repeats <- users$max_repeats >= 4

# Taking a look at the users with too many repeats
users[users$too_many_repeats,]

,id,user_name,password,length,too_short,common_password,common_word,first_name,last_name,uses_name,max_repeats,too_many_repeats
,<int>,<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<lgl>,<int>,<lgl>
147,147,patti.dixon,555555,6,TRUE,TRUE,FALSE,patti,dixon,FALSE,6,TRUE
573,573,cornelia.bradley,555555,6,TRUE,TRUE,FALSE,cornelia,bradley,FALSE,6,TRUE
645,645,essie.lopez,11111,5,TRUE,TRUE,FALSE,essie,lopez,FALSE,5,TRUE
799,799,charley.key,888888,6,TRUE,TRUE,FALSE,charley,key,FALSE,6,TRUE
808,808,thurman.osborne,rinnnng0,8,FALSE,FALSE,FALSE,thurman,osborne,FALSE,4,TRUE
942,942,mitch.ferguson,aaaaaa,6,TRUE,TRUE,FALSE,mitch,ferguson,FALSE,6,TRUE


##  All together now!
<p>Now we have implemented all the basic tests for bad passwords suggested by NIST Special Publication 800-63B! What's left is just to flag all bad passwords and maybe to send these users an e-mail that strongly suggests they change their password.</p>

In [67]:
# Flagging all passwords that are bad
users$bad_password <- 
    users$too_short | 
    users$common_password |
    users$common_word |
    users$uses_name |
 users$too_many_repeats


# Counting the number of bad passwords
sum(users$bad_password)
# Looking at the first 100 bad passwords
head(users$password[users$bad_password],100)

[1] 419

[1] "5278049"      "master"       "murphy"       "lwsves2"      "hubbard"     
  [6] "310356"       "oZ4k0QE"      "chelsea"      "zvc1939"      "nickgd"      
 [11] "cocacola"     "woodard"      "AJ9Da"        "ewokzs"       "YyGjz8E"     
 [16] "reid"         "jOYZBs8"      "wwewwf1"      "225377"       "NdZ7E6"      
 [21] "CQB3Z"        "diffo"        "123456789"    "y8uM7D6"      "mikeloo"     
 [26] "golden"       "apr2502"      "jfasii"       "sanaang"      "jecarn"      
 [31] "dakami"       "thx1138"      "midnight"     "7xFHcdy"      "dorian"      
 [36] "2458098"      "hurley"       "UYHUGG"       "wishes"       "jake145"     
 [41] "Oli#2b"       "monkey"       "shy!"         "chifito"      "123ASH"      
 [46] "juliette"     "kokjoon"      "crystal"      "scanner"      "andrew"      
 [51] "caleb"        "dreker"       "h3ZFd"        "replaced"     "buster"      
 [56] "nu"           "ware"         "46ofus"       "rr55367"      "fastman"     
 [61] "biodiversity" "555555"       "lara"         "9786435"      "roland"      
 [66] "gokuass"      "ov0moh"       "stefan"       "9812684"      "TIRO02"      
 [71] "dixie"        "gtapbt"       "123654"       "1036900"      "flavor"      
 [76] "FGPb9gX"      "vHO2y"        "precise"      "190664"       "4f3aRNG"     
 [81] "austin"       "jtc123"       "eLvtn"        "122412"       "8309113"     
 [86] "dillard"      "william"      "NCAA50"       "4101516"      "leyvi"       
 [91] "ancan"        "blake"        "tdutza"       "framework"    "golfer"      
 [96] "winter"       "groan1"       "pharmacology" "bs4519"       "prince"

##  Otherwise, the password should be up to the user
<p>In this notebook, we've implemented the password checks recommended by the NIST Special Publication 800-63B. It's certainly possible to better implement these checks, for example, by using a longer list of common passwords. Also note that the NIST checks in no way guarantee that a chosen password is good, just that it's not obviously bad.</p>
<p>Apart from the checks we've implemented above the NIST is also clear with what password rules should <em>not</em> be imposed:</p>
<blockquote>
  <p>Verifiers SHOULD NOT impose other composition rules (e.g., requiring mixtures of different character types or prohibiting consecutively repeated characters) for memorized secrets. Verifiers SHOULD NOT require memorized secrets to be changed arbitrarily (e.g., periodically).</p>
</blockquote>
<p>So the next time a website or app tells you to "include both a number, symbol and an upper and lower case character in your password" you should send them a copy of <a href="https://pages.nist.gov/800-63-3/sp800-63b.html">NIST Special Publication 800-63B</a>.</p>

In [69]:
# Enter a password that passes the NIST requirements
# PLEASE DO NOT USE AN EXISTING PASSWORD HERE
new_password <- "iamnobody00"